This is a notebook for a simple traditional IDMRG algorithm with block growth method. 

When writing the julia code shown here, I refer to the python code written by Dr. Garrison and Dr. Mishmash from the website:
    
    https://simple-dmrg.readthedocs.io/en/latest/
    
In the following code, I take U(1) symmetry of the system into consideration. In this case, we can divide the Hilbert space into several parts and deal with them seperately, which will enhance the computational efficiency.

The key point for good-quantum-number-conserve DMRG is setting up the mapping, 
        
          good quantum number => Hamiltonian block.
          
With the mapping, we can extract the non-zero part from the total superblock Hamiltonian matrix, in order to reduce the size of the matrix. We can also diagonalize the density matrix blocks seperately, which will also reduce the computating time.

In the following code, we focus on finite system. We firstly gradually enlarge the system part and the environment part until reaching the total size of the system (similar to IDMRG). Then, we need to do sweep until reaching convergence.

In [1]:
using LinearAlgebra
using KrylovKit
using SparseArrays
Sz = sparse([0.5 0;0 -0.5])
Sp = sparse([0 1; 0 0])
Sm = sparse([0 0; 1 0])
phy_d = 2
single_site_line_to_qn = [0.5 -0.5]
struct block
    len ::Int
    basis_size ::Int
    ops ::Vector{Any}
    line_to_qn
end
function outer_sum_flat(a,b)
    c = kron(b,ones(length(a)))+kron(ones(length(b))',a')
    c = reshape(c',1,(size(c,1)*size(c,2)))
    return c
end
function is_valid_block(Block)
    @assert length(Block.line_to_qn) == Block.basis_size
    for op in Block.ops
        @assert size(op) == (Block.basis_size,Block.basis_size)
    end
    return true
end
function two_sites_enlarge(Sz1,Sp1,Sm1,Sz2,Sp2,Sm2)
    J = 1
    Jz = 1
    TwoSiteH = (J/2)*kron(Sp1,Sm2)+(J/2)*kron(Sm1,Sp2)+Jz*kron(Sz1,Sz2)
    return TwoSiteH
end
initial_block = block(1,phy_d,[spzeros(phy_d,phy_d),Sz,Sp,Sm],single_site_line_to_qn)
function enlarge_block(b) #b is a block
    mblock = b.basis_size
    o = b.ops
    H = kron(o[1],sparse(I,phy_d,phy_d))+kron(sparse(I,mblock,mblock),zeros(phy_d,phy_d))
    H = H + two_sites_enlarge(o[2],o[3],o[4],Sz,Sp,Sm)
    conn_Sz = kron(sparse(I,mblock,mblock),Sz)
    conn_Sp = kron(sparse(I,mblock,mblock),Sp)
    conn_Sm = kron(sparse(I,mblock,mblock),Sm)
    enlarged_line_to_qn = outer_sum_flat(b.line_to_qn,single_site_line_to_qn)
    return block(b.len+1,b.basis_size*phy_d,[H,conn_Sz,conn_Sp,conn_Sm],enlarged_line_to_qn)
end
function ChangeBasis(operator,trans_mat)
    op_new = trans_mat'*operator*trans_mat
    return op_new
end
function index_map(arr)
    d = Dict()
    for i in enumerate(arr)
        if haskey(d,i[2])
            append!(d[i[2]],i[1])
        else
            d[i[2]] = []
            append!(d[i[2]],i[1])
        end
    end
    return d
end
function single_dmrg_step(sys,env,m,target_qn) #m is the bond dimension cutoff
    @assert is_valid_block(sys)
    @assert is_valid_block(env)
    sys_enl = enlarge_block(sys)
    sys_enl_qn_to_line = index_map(sys_enl.line_to_qn)
    if sys == env
        env_enl = sys_enl
        env_enl_qn_to_line = sys_enl_qn_to_line
    else
        env_enl = enlarge_block(env)
        env_enl_qn_to_line = index_map(env_enl.line_to_qn)
    end
    @assert is_valid_block(sys_enl)
    @assert is_valid_block(env_enl)
    
    m_sys_enl = sys_enl.basis_size
    m_env_enl = env_enl.basis_size
    sys_enl_op = sys_enl.ops
    env_enl_op = env_enl.ops
    superblock_H = kron(sys_enl_op[1],sparse(I,m_env_enl,m_env_enl))+kron(sparse(I,m_sys_enl,m_sys_enl),env_enl_op[1])+two_sites_enlarge(sys_enl_op[2],sys_enl_op[3],sys_enl_op[4],env_enl_op[2],env_enl_op[3],env_enl_op[4])
    
    
    sys_enl_qn_to_newbasisline = Dict()#enlarged_system_good_qn=>index of non-zero Hamiltonian block
    non_zero_superblock_line = [] # all the super_block_hamiltonian lines satisfy good quantum number conservation
    for qn_state in sys_enl_qn_to_line
        sys_enl_qn = qn_state[1]
        sys_enl_qn_to_newbasisline[sys_enl_qn] = []
        env_enl_qn = target_qn - sys_enl_qn
        if env_enl_qn in keys(env_enl_qn_to_line)
            for i in qn_state[2]
                i_offset = m_env_enl*(i-1)
                for j in env_enl_qn_to_line[env_enl_qn]
                    current_index = length(non_zero_superblock_line)+1
                    append!(sys_enl_qn_to_newbasisline[sys_enl_qn],current_index)
                    append!(non_zero_superblock_line,i_offset+j)
                end
            end
        end
    end
    qn_conserve_superblock_H = superblock_H[:,non_zero_superblock_line][non_zero_superblock_line,:]
    # take the qn conserve part of the total superblock Hamiltonian
    # sys_enl_qn_to_newbasisline record: enlarged system qn => lines in the new Hamiltonian
    vals,vecs,info = KrylovKit.eigsolve(qn_conserve_superblock_H,1,:SR)
    energy = vals[1]
    groundstate = vecs[1]
    qn_to_rho_block = Dict()
    for qn_state in sys_enl_qn_to_newbasisline
        if !(isempty(qn_state[2]))
            psi0_sector = groundstate[qn_state[2]]
            l1 = length(env_enl_qn_to_line[qn_state[1]])
            l2 = length(sys_enl_qn_to_line[qn_state[1]])
            l_env = length(psi0_sector)÷(length(sys_enl_qn_to_line[qn_state[1]]))
            psi0_sector = reshape(psi0_sector,(l_env,length(sys_enl_qn_to_line[qn_state[1]])))
            psi0_sector = transpose(psi0_sector)
            qn_to_rho_block[qn_state[1]] = Hermitian(psi0_sector*psi0_sector')
        end
    end
    #qn_to_rho_block: enlarged system qn=>density matrix block
    possible_eigenstates = []
    for qn_rho in qn_to_rho_block
        sol = eigen(qn_rho[2])
        evector = [sol.vectors[:,i] for i = 1:length(sol.values)]
        evalue = sol.values
        current_lines = sys_enl_qn_to_line[qn_rho[1]]
        # the enlarged system block line indices
        # because we need to do basis transformation
        for i in zip(evalue,evector)
            append!(possible_eigenstates,[(i[1],i[2],qn_rho[1],current_lines)])
        end
    end
    sort!(possible_eigenstates,rev=true,by = x->x[1])
    #possible eigenstates: [density,density matrix eigenvector,system part qn,enlarged system block lines]
    mm = min(m,length(possible_eigenstates))
    trans_mat = zeros(ComplexF64,m_sys_enl,mm)
    new_line_to_qn = zeros(mm)
    for i in enumerate(possible_eigenstates[1:mm])
        for j in zip(i[2][4],i[2][2])#zip(enlarged system block lines,block rho matrix eigenvectors)
            trans_mat[j[1],i[1]] = j[2]
            #trans_mat[:,i] = [:] only the block index need to be filled
        end
        new_line_to_qn[i[1]] = i[2][3]
    end
    truncation_error = 1-sum([x[1] for x in possible_eigenstates[1:mm]])
    println("truncation error = $truncation_error")
    new_length = sys_enl.len
    new_size = mm
    new_H = ChangeBasis(sys_enl_op[1],trans_mat)
    new_sz = ChangeBasis(sys_enl_op[2],trans_mat)
    new_sp = ChangeBasis(sys_enl_op[3],trans_mat)
    new_sm = ChangeBasis(sys_enl_op[4],trans_mat)
    new_line_to_qn = reshape(new_line_to_qn,1,length(new_line_to_qn))
    new_block = block(new_length,new_size,[new_H,new_sz,new_sp,new_sm],new_line_to_qn)
    return new_block,energy
end
function graphic(sys_block,env_block;sys_label="l")
    @assert sys_label in ["l","r"]
    graphic = string(repeat("=",sys_block.len),"**",repeat("-",env_block.len))
    if sys_label =="r"
        graphic = reverse(graphic)
    end
    return graphic
end
function infinite_system(L,m,target_qn)
    b = initial_block
    while 2*(b.len) <L
        current_L = 2*b.len+2
        current_target_qn = Int(target_qn)*current_L÷L
        println("L = $current_L")
        b,energy = single_dmrg_step(b,b,m,current_target_qn)
        println("E/L = $(energy/current_L)")
    end
end

function finite_system_algorithm(L,m_warmup,m_sweep_list,target_qn)
    @assert L%2 ==0 #require that L is an even number
    b = initial_block
    l_block = Dict(b.len=>b)
    r_block = Dict(b.len=>b)
    while 2*b.len <L
        print(graphic(b,b))
        current_L = 2*b.len+2
        current_target_qn = Int(target_qn)*current_L÷L
        println("L = $current_L")
        b,energy = single_dmrg_step(b,b,m_warmup,current_target_qn)
        println("E/L = $(energy/current_L)")
        l_block[b.len] = b
        r_block[b.len] = b
    end
    # Now we have got the initial state
    sys_label,env_label = "l","r"
    sys_block = b
    for m in m_sweep_list
        while true
            if env_label == "r"
                env_block = r_block[L-sys_block.len-2]
            else
                env_block = l_block[L-sys_block.len-2]
            end
            if env_block.len ==1
                sys_block,env_block = env_block,sys_block
                sys_label,env_label = env_label,sys_label
            end
            print(graphic(sys_block,env_block;sys_label))
            sys_block,energy = single_dmrg_step(sys_block,env_block,m,target_qn)
            println("E/L = $(energy/L)")
            if sys_label == "r"
                r_block[sys_block.len] = sys_block
            else
                l_block[sys_block.len] = sys_block
            end
            if sys_label=="l" && 2*(sys_block.len)==L
                break
            end
        end
    end
end
finite_system_algorithm(20, 10, [10,20,30,40,40],0)


=**-L = 4
truncation error = 0.0
E/L = -0.4040063509461093
==**--L = 6
truncation error = 8.326672684688674e-15
E/L = -0.41559618898131917 - 7.390263700547338e-18im
===**---L = 8
truncation error = 3.4092688017128836e-7
E/L = -0.42186657483598905 - 1.2501509836414696e-17im
====**----L = 10
truncation error = 1.4085888766501853e-7
E/L = -0.4258028861280815 - 2.461557927920675e-18im
=====**-----L = 12
truncation error = 2.1600238701013907e-6
E/L = -0.42850647272181486 + 9.562880224406843e-18im
======**------L = 14
truncation error = 9.7056440484522e-7
E/L = -0.4304772948659509 + 1.0253361453528303e-18im
=======**-------L = 16
truncation error = 5.544294883086387e-6
E/L = -0.431979381832864 + 1.1401828339898494e-17im
========**--------L = 18
truncation error = 2.533005380889186e-6
E/L = -0.4331599020425138 + 1.0119066047662112e-17im
=========**---------L = 20
truncation error = 9.905730359971443e-6
E/L = -0.43411452436306125 + 5.893148176164639e-18im
==========**--------truncation error =

=============**-----truncation error = 1.625799495030833e-11
E/L = -0.4341236534573416 + 1.5596112535644687e-17im
==============**----truncation error = 3.637268264355953e-11
E/L = -0.43412365383071644 - 5.261497018982338e-18im
===============**---truncation error = -2.6645352591003757e-15
E/L = -0.43412365382127815 + 1.3141775455406923e-17im
================**--truncation error = -1.9984014443252818e-15
E/L = -0.4341236538212792 + 3.407880297210147e-17im
-----------------**=truncation error = -3.774758283725532e-15
E/L = -0.4341236538212779 + 6.335678623652363e-17im
----------------**==truncation error = 1.887379141862766e-15
E/L = -0.4341236538212755 + 2.565946729840522e-18im
---------------**===truncation error = 1.887379141862766e-15
E/L = -0.4341236538212755 + 2.0821655519731926e-17im
--------------**====truncation error = 3.637223855434968e-11
E/L = -0.43412365383071727 - 1.371456965038476e-17im
-------------**=====truncation error = 1.9578227927752323e-11
E/L = -0.43412365382556

------------**======truncation error = 6.334821556208681e-12
E/L = -0.4341236667015235 + 1.1866104309889023e-17im
-----------**=======truncation error = 1.2029044427208646e-11
E/L = -0.4341236667028678 + 6.2415471517410995e-18im
----------**========truncation error = 2.0390800159475475e-11
E/L = -0.4341236667040529 + 4.280134377030538e-17im
---------**=========truncation error = 1.885214206964747e-11
E/L = -0.43412366670424285 + 2.393046425778457e-17im
--------**==========truncation error = 2.0392909583222263e-11
E/L = -0.4341236667040514 + 2.6124736224196896e-16im
-------**===========truncation error = 1.2020273665314107e-11
E/L = -0.4341236667028706 + 1.741949706130503e-17im
------**============truncation error = 6.341593916658894e-12
E/L = -0.43412366670152275 + 6.7027579824382475e-18im
-----**=============truncation error = 2.2604140781368187e-13
E/L = -0.43412366670037594 - 5.936291371280354e-18im
----**==============truncation error = -8.881784197001252e-15
E/L = -0.4341236667003